# Land Parcel Identification System (LPIS) prediction for Slovenia

This notebook shows the steps towards constructing machine learning model for LPIS prediction for Slovenia.

### Overview

#### Requirements
1. Downloaded and processed Sentinel data *(relevant [notebook](https://github.com/sentinel-hub/eo-learn/blob/master/examples/land-cover-map/SI_LULC_pipeline.ipynb))*
    * Sentinel-2 data download
    * cloud detection and masking
    * interpolation    
    

2. Downloaded and grouped LPIS data *(relevant [notebook](LPISDataFromGeopedija.ipynb))*
    * LPIS data download
    * LPIS class grouping 
    
#### Samples construction
1. Data sample construction
    * edge mask construction
    * oversampling
2. Feature calculation
    * stream feature calculation
    * elevation
    
#### Feature selection and model construction
1. Feature selection
    * FASTENER
2. Model construction
    * data normalization
    * model training
    * model testing
3. Model usage
    * prediction of LPIS on chosen region



In [ ]:
# Firstly, some necessary imports
import os
import numpy as np

seed = 42
np.random.seed(seed)

import matplotlib.pyplot as plt

# from eolearn.mask import EdgeExtractionTask
# from eolearn.geometry import BalancedClassSampler, BalancedClassSamplerTask
from notebook_temporary.edge_extraction import EdgeExtractionTask  # Change once it will be in develop
from notebook_temporary.sampling import BalancedClassSampler, \
    BalancedClassSamplerTask  # Change once it will be in develop

from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, \
    LoadTask, SaveTask, EOExecutor
from eolearn.io import SentinelHubDemTask


## Samples construction

### Edge mask calculation
When training the classifier we don't want to include the pixels on the borders of parcels. These pixels are potential mixed-class instances that can have a negative effect on the learning process. So prior to sampling we will construct an timeless mask which excludes the edges. This is already done in an EOTask so we just need to call it.

Since we will be classificating crops we will calculate edges based on the NDVI metric and the green band. Along those, let's calculate all the metrics that we will be needing later which are the other base bands red, blue, NIR (Near infra red) and the vegetation related indices NIR, ARVI, EVI, NDVI, NDWI, SIPI and SAVI.

In [ ]:
class AddBaseFeatures(EOTask):

    def __init__(self, c1=6, c2=7.5, L=1, Lvar=0.5, delta=10 ** -10):
        self.c1 = c1
        self.c2 = c2
        self.L = L
        self.Lvar = Lvar

        # We add a small number that doesn't significantly change the result to avoid divisions by zero
        self.delta = delta

    def execute(self, eopatch):
        nir = eopatch.data['BANDS'][..., [7]]
        blue = eopatch.data['BANDS'][..., [1]]
        red = eopatch.data['BANDS'][..., [3]]
        eopatch.add_feature(FeatureType.DATA, 'NIR', nir)

        arvi = np.clip((nir - (2 * red) + blue) / (nir + (2 * red) + blue + self.delta), -1, 1)
        eopatch.add_feature(FeatureType.DATA, 'ARVI', arvi)

        evi = np.clip(2.5 * ((nir - red) / (nir + (self.c1 * red) - (self.c2 * blue) + self.L + self.delta)), -1, 1)
        eopatch.add_feature(FeatureType.DATA, 'EVI', evi)

        ndvi = np.clip((nir - red) / (nir + red + self.delta), -1, 1)
        eopatch.add_feature(FeatureType.DATA, 'NDVI', ndvi)

        ndwi = np.clip((blue - red) / (blue + red + self.delta), -1, 1)
        eopatch.add_feature(FeatureType.DATA, 'NDWI', ndwi)

        sipi = np.clip((nir - blue) / (nir - red + self.delta), 0, 2)
        eopatch.add_feature(FeatureType.DATA, 'SIPI', sipi)

        savi = np.clip(((nir - red) / (nir + red + self.Lvar + self.delta)) * (1 + self.Lvar), -1, 1)
        eopatch.add_feature(FeatureType.DATA, 'SAVI', savi)

        return eopatch

In [ ]:
base = AddBaseFeatures()
edges = EdgeExtractionTask(features={FeatureType.DATA: ['NDVI', 'GREEN']})

# This tutorial assumes all the patches are saved in current directory in folder patches. You can change this here
patches_path = f'{os.path.abspath(os.getcwd())}/patches'
# save_patch_location = patches_path
# save_path = f'{os.path.abspath(os.getcwd())}/patches_output'
save_path = patches_path
load = LoadTask(patches_path)
if not os.path.isdir(save_path):
    os.makedirs(save_path)
save = SaveTask(save_path, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

execution_args = []
for name in next(os.walk(patches_path))[1]:
    execution_args.append({
        load: {'eopatch_folder': name},
        save: {'eopatch_folder': name}
    })

workflow = LinearWorkflow(load,
                          base,
                          edges,
                          save)

executor = EOExecutor(workflow, execution_args)
executor.run(multiprocess=True)

In [ ]:
# Visualize the mask

patch_name = next(os.walk(patches_path))[1][0]
eopatch = EOPatch.load(f'{save_path}/{patch_name}')

edges = eopatch.mask_timeless['EDGES_INV'].squeeze()
img = np.clip(eopatch.data['BANDS'][10][..., [3, 2, 1]] * 3.5, 0, 1)

plt.figure(figsize=(18, 9))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(edges, cmap='gray')
plt.show()

### Sampling
We don't want to train a classifier on every point as earth observation data can be very large and thus unfeasible. In this step we will choose a sample of the area. The properties of a sample should be that it is taken from all region uniformly and because we are training a classifier we want to have each class represented equally. This can be done with the BalancedClassSampler task. We will also exclude the the edge regions we calculated before to have a "clean" sample.

In [ ]:
class_feature = (FeatureType.MASK_TIMELESS, 'LPIS_2017')
sampler = BalancedClassSampler(class_feature=class_feature,
                               valid_mask=(FeatureType.MASK_TIMELESS, 'EDGES_INV'),
                               seed=seed)
sampler.sample_folder(save_path, lazy_loading=True)
distribution = sampler.get_prior_class_distribution()

# sorting so we can easier see which classes are less represented
distribution = {k: v for k, v in sorted(distribution.items(), key=lambda item: item[1])}

# Lets display what was sampled
print(distribution)

# We can try to increase the smallest samples classes by sampling again and this time specifying which classes
# are weak to do additional sampling around them once encountered
sampler2 = BalancedClassSampler(class_feature=class_feature,
                               valid_mask=(FeatureType.MASK_TIMELESS, 'EDGES_INV'),
                               seed=seed,
                               weak_classes=[6, 7, 5, 3])
sampler2.sample_folder(save_path, lazy_loading=True)
distribution2 = sampler2.get_prior_class_distribution()
distribution2 = {k: v for k, v in sorted(distribution2.items(), key=lambda item: item[1])}
print(distribution2)

# Final samples
samples = sampler2.get_balanced_data()
print(samples)

## Stream features calculation
As we didn't have stream features calculated for all data we will now compute neccesary stream features for only the points that we decided on with sampling in previous step.
We will also add height of the pixel as one of the features.

In [ ]:
# Assumes all the patches have same shape
height, width, _ = eopatch[class_feature].shape

# For each patch that contains any samples we will construct mask to specify where the stream
# features are computer. We don't want to compute them for the whole patch as it would take a long time
patches = next(os.walk(patches_path))[1]

# Separate points by patch. Patch identifier is by default it's folder name
separated_by_patch = [(x, samples[samples['patch_identifier'] == x]) for x in patches]
execution_args = []
for name, points in separated_by_patch:
    eopatch = EOPatch.load(f'{save_path}/{name}')
    stream_mask = np.zeros((width, height))
    for x, y in zip(points['x'], points['y']):
        stream_mask[x, y] = True
    eopatch.add_feature(FeatureType.MASK_TIMELESS, 'STREAM_VALID', stream_mask[..., np.newaxis])
    eopatch.save(f'{save_path}/{name}', eopatch, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

In [ ]:
###### TODO Filip
# Skopiraj kodo ko bo v `notebook_temporary` folder
spatio_temporal = AddSpatioTemporalFeaturesTask()
max_mean_len = MaxMeanLenTask()
surface_extraction = SurfaceExtractionTask()
max_min_temporal = AddMaxMinTemporalIndicesTask()
max_min_ndvi_slope = AddMaxMinNDVISlopeIndicesTask()
all_stream_feature_names = []  # 'NDVI_max_mean', 'NDVI_slope' .....
##### TODO Filip

execution_args = []
for name in next(os.walk(patches_path))[1]:
    execution_args.append({
        load: {'eopatch_folder': name},
        save: {'eopatch_folder': name}
    })
    
# This tasks adds elevation data to the patch
dem = SentinelHubDemTask((FeatureType.DATA_TIMELESS, 'DEM'), size=(height, width))
workflow = LinearWorkflow(load,
                          dem,
                          spatio_temporal,
                          max_mean_len,
                          surface_extraction,
                          max_min_temporal,
                          max_min_ndvi_slope,
                          save)
executor = EOExecutor(workflow, execution_args)
executor.run(multiprocess=True)

In [ ]:
# Once the stream features are calculated we just need to put them into the format suitable for feature selection
# we need to contruct a pandas DataFrame with column names of features and the class name. Each row represents a single point
extended_samples = []
for name, points in separated_by_patch:
    eopatch = EOPatch.load(f'{save_path}/{name}', lazy_loading=True)
    for x, y in zip(points['x'], points['y']):
        point_data = [(class_feature[1], eopatch[class_feature][x, y, 0])] \
                     + [(f, eopatch.data_timeless[f][x, y].squeeze()) for f in all_stream_feature_names]
        extended_samples.append(dict(point_data))
extended_samples = pd.DataFrame(extended_samples)

## Feature selection and model construction
The features will be chosen using the FASTENER algorithm
## TODO filip

In [ ]:
selected_features = FASTENER(extended_samples) ###### TODO FILIP

### Model construction
Once we have final samples with selected only the best features we can train a model.

In [ ]:
 model = tree.DecisionTreeClassifier()
    y = extended_samples['LPIS_2017'].to_numpy()
    x = extended_samples[selected_features].to_numpy()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)

    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    labels = Counter(extended_samples['LPIS_2017']).keys()
    no_classes = range(len(labels))
    fig, ax = plt.subplots()
    ax.set_ylim(bottom=0.14, top=0)
    plot_confusion_matrix(model, x_test, y_test, labels=no_classes,
                          display_labels=labels,
                          cmap='viridis',
                          include_values=False,
                          xticks_rotation='vertical',
                          normalize='pred',
                          ax=ax)
    f1 = f1_score(y_test, y_pred, labels=no_classes, average='macro')
    print(f'f1: {f1}')
    plt.show()

### Model usage on a sample region